In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
df1 = pd.read_csv('Bengaluru_House_Data.csv')

In [ ]:
df1

In [ ]:
df1 = df1.rename(columns = {'size' : 'rooms'})
df1.dropna(subset = ['rooms'], inplace=True)
df1['bedrooms'] = df1.rooms.str.split(' ', expand=True)[0]
df1['kitchen'] = df1.rooms.str.contains('K')*1
df1['hall'] = df1.rooms.str.contains('H')*1
df1.drop('rooms', axis=1, inplace=True)
df1 = df1.astype({'bedrooms':'int64'})

In [ ]:
lower_upper_val = df1.total_sqft[df1.total_sqft.str.contains(' - ')].str.split(' - ', expand = True).astype('float64')
mean_val_sqft = lower_upper_val.assign(mean_val = (lower_upper_val[0]+lower_upper_val[1])/2).mean_val
df1.total_sqft[df1.total_sqft.str.contains(' - ')] = mean_val_sqft

In [ ]:
area_conversion_to_sqft = {
    'Sq. Yards' : 9,
    'Sq. Meter' : 10.76,
    'Acres' : 43560,
    'Cents' : 435.56,
    'Guntha' : 1089,
    'Grounds' : 2400.35,
    'Perch' : 272.25
}

In [ ]:
area_to_convert = df1.total_sqft[df1.total_sqft.str.contains('[a-zA-Z]') == True]
area_values = area_to_convert.str.extract('([\d\.]+)').astype('float64')[0]
conversion_factor = df1.total_sqft.str.extract('([\D]+$)').dropna()[0].replace(area_conversion_to_sqft)
converted_area = round(area_values*conversion_factor, 2)

In [ ]:
df1.info()

In [ ]:
df2 = df1.copy()
df2.total_sqft[converted_area.index] = converted_area
df2.total_sqft = df2.total_sqft.astype('float64').astype('int')

In [ ]:
df2.dropna(subset=['bath'], inplace=True)

In [ ]:
df2.bath = df2.bath.astype('int')

In [ ]:
df2.drop(['society'],1,inplace=True)

In [ ]:
df2.balcony.fillna(0, inplace=True)

In [ ]:
df2.rename(columns = {'total_sqft' : 'area'}, inplace=True)

In [ ]:
df2.rename(columns = {'bath' : 'bathrooms'}, inplace=True)

In [ ]:
df2.rename(columns = {'balcony' : 'numbalcony'}, inplace=True)

In [ ]:
df2['balcony'] = (df2.numbalcony > 0)

In [ ]:
df2.drop(['numbalcony'],1, inplace=True)

In [ ]:
df2.info()
df2.describe()

In [ ]:
df2.drop(['availability', 'kitchen', 'hall', 'location'], 1, inplace=True)

In [ ]:
df2.rename(columns = {'area_type' : 'home_type'}, inplace=True)

In [ ]:
for index, row in df2.iterrows():
  df2.at[index, 'price'] *= 100000

In [ ]:
df2['home_type'] = df2['home_type'].replace({'Super built-up  Area': 'Condos/Co-ops', 'Built-up  Area': 'Apartments', 'Plot  Area':'Houses', 'Carpet  Area':'Multi-family'})

In [ ]:
df2 = df2[df2['price'] < 100000000].reset_index(drop=True)

In [ ]:
df2.to_csv('Property_Data.csv', index=False)